In [1]:
import pandas as pd
import gradio as gr
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors

# Excel file path
file_path = "restaurant_warehouse.xlsx"

# Load or create inventory
try:
    df = pd.read_excel(file_path)
except FileNotFoundError:
    df = pd.DataFrame(columns=["Item ID", "Item Name", "Category", "Quantity", "Unit", "Price per Unit", "Total Cost", "Supplier", "Expiry Date"])
    df.to_excel(file_path, index=False)

# Ensure sorted Item IDs at startup
df = df.sort_values(by="Item ID").reset_index(drop=True)

# Function to show inventory
def show_inventory():
    return df

# Function to add a product
def add_product(item_id, name, category, quantity, unit, price, supplier, expiry):
    global df
    if not all([item_id, name, category, quantity, unit, price, supplier, expiry]):
        return "All fields are required!"
    
    new_data = {
        "Item ID": item_id,
        "Item Name": name,
        "Category": category,
        "Quantity": int(quantity),
        "Unit": unit,
        "Price per Unit": float(price),
        "Total Cost": int(quantity) * float(price),
        "Supplier": supplier,
        "Expiry Date": expiry
    }
    
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True).sort_values(by="Item ID").reset_index(drop=True)
    df.to_excel(file_path, index=False)
    return df

# Function to modify a product dynamically
def get_item_ids():
    return df["Item ID"].tolist()

def get_columns():
    return ["Item Name", "Category", "Quantity", "Unit", "Price per Unit", "Supplier", "Expiry Date"]

def modify_product(item_id, column, new_value):
    global df
    if item_id in df["Item ID"].values:
        df.loc[df["Item ID"] == item_id, column] = new_value
        if column in ["Quantity", "Price per Unit"]:
            df["Total Cost"] = df["Quantity"].astype(float) * df["Price per Unit"].astype(float)
        df.to_excel(file_path, index=False)
        return df
    return "Item not found"

# Function to remove a product
def remove_product(item_id):
    global df
    df = df[df["Item ID"] != item_id].sort_values(by="Item ID").reset_index(drop=True)
    df.to_excel(file_path, index=False)
    return df

# Function to search products
def search_product(name):
    return df[df["Item Name"].str.contains(name, case=False, na=False)]

# Function to sort by price
def sort_by_price():
    return df.sort_values(by="Price per Unit")

# Function to identify low stock items
def low_stock_alert(threshold=10):
    return df[df["Quantity"] < threshold]

# Function to export data to PDF
def generate_pdf():
    pdf_file = "inventory_report.pdf"
    doc = SimpleDocTemplate(pdf_file, pagesize=letter)
    elements = []

    # Define table headers
    data = [["Item ID", "Item Name", "Category", "Quantity", "Unit", "Price per Unit", "Total Cost", "Supplier", "Expiry Date"]]

    # Add inventory data
    for _, row in df.iterrows():
        data.append([
            row["Item ID"], row["Item Name"], row["Category"], row["Quantity"], row["Unit"],
            row["Price per Unit"], row["Total Cost"], row["Supplier"], row["Expiry Date"]
        ])

    # Create table with styles
    table = Table(data)
    table.setStyle(TableStyle([
        ("BACKGROUND", (0, 0), (-1, 0), colors.grey),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
        ("ALIGN", (0, 0), (-1, -1), "CENTER"),
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("BOTTOMPADDING", (0, 0), (-1, 0), 10),
        ("GRID", (0, 0), (-1, -1), 1, colors.black)
    ]))

    elements.append(table)
    doc.build(elements)

    return pdf_file

# Gradio UI with Tabs
with gr.Blocks() as demo:
    gr.Markdown("# 🏢 Warehouse Management System")
    
    with gr.Tabs():
        with gr.Tab("📋 Inventory"):
            show_button = gr.Button("Show Inventory")
            inventory_output = gr.Dataframe()
            show_button.click(show_inventory, outputs=inventory_output)

        with gr.Tab("➕ Add Product"):
            item_id = gr.Textbox(label="Item ID")
            name = gr.Textbox(label="Item Name")
            category = gr.Textbox(label="Category")
            quantity = gr.Number(label="Quantity")
            unit = gr.Textbox(label="Unit")
            price = gr.Number(label="Price per Unit")
            supplier = gr.Textbox(label="Supplier")
            expiry = gr.Textbox(label="Expiry Date")
            add_button = gr.Button("Add Product")
            add_output = gr.Dataframe()
            add_button.click(add_product, inputs=[item_id, name, category, quantity, unit, price, supplier, expiry], outputs=add_output)

        with gr.Tab("🔄 Modify Product"):
            item_id_dropdown = gr.Dropdown(label="Select Item ID", choices=get_item_ids())
            column_dropdown = gr.Dropdown(label="Select Column to Modify", choices=get_columns())
            new_value_input = gr.Textbox(label="Enter New Value")
            mod_button = gr.Button("Modify")
            mod_output = gr.Dataframe()
            mod_button.click(modify_product, inputs=[item_id_dropdown, column_dropdown, new_value_input], outputs=mod_output)

        with gr.Tab("❌ Remove Product"):
            remove_id = gr.Dropdown(label="Select Item ID", choices=get_item_ids())
            remove_button = gr.Button("Remove")
            remove_output = gr.Dataframe()
            remove_button.click(remove_product, inputs=[remove_id], outputs=remove_output)

        with gr.Tab("🔎 Search Product"):
            search_name = gr.Textbox(label="Search by Name")
            search_button = gr.Button("Search")
            search_output = gr.Dataframe()
            search_button.click(search_product, inputs=[search_name], outputs=search_output)

        with gr.Tab("💰 Sort by Price"):
            sort_button = gr.Button("Sort")
            sort_output = gr.Dataframe()
            sort_button.click(sort_by_price, outputs=sort_output)

        with gr.Tab("⚠ Low Stock Alert"):
            threshold = gr.Number(label="Threshold (Default: 10)", value=10)
            low_stock_button = gr.Button("Check Low Stock")
            low_stock_output = gr.Dataframe()
            low_stock_button.click(low_stock_alert, inputs=[threshold], outputs=low_stock_output)

        with gr.Tab("📄 Export to PDF"):
            pdf_button = gr.Button("Download PDF")
            pdf_output = gr.File()
            pdf_button.click(generate_pdf, outputs=pdf_output)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [2]:
import pandas as pd
import gradio as gr
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors

# Excel file path
file_path = "restaurant_warehouse.xlsx"

# Load or create inventory
try:
    df = pd.read_excel(file_path)
except FileNotFoundError:
    df = pd.DataFrame(columns=["Item ID", "Item Name", "Category", "Quantity", "Unit", "Price per Unit", "Total Cost", "Supplier", "Expiry Date"])
    df.to_excel(file_path, index=False)

# Ensure sorted Item IDs at startup
df = df.sort_values(by="Item ID").reset_index(drop=True)

# Function to show inventory
def show_inventory():
    return df

# Function to add a product
# Function to add a product (ensuring Item IDs remain sorted)
def add_product(item_id, name, category, quantity, unit, price, supplier, expiry):
    global df
    if not all([item_id, name, category, quantity, unit, price, supplier, expiry]):
        return "All fields are required!"
    
    new_data = {
        "Item ID": item_id,
        "Item Name": name,
        "Category": category,
        "Quantity": int(quantity),
        "Unit": unit,
        "Price per Unit": float(price),
        "Total Cost": int(quantity) * float(price),
        "Supplier": supplier,
        "Expiry Date": expiry
    }
    
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
    
    # 🔹 Ensure sorting by Item ID
    df = df.sort_values(by="Item ID", ascending=True).reset_index(drop=True)

    df.to_excel(file_path, index=False)
    return df


# Function to modify a product dynamically
def get_item_ids():
    return df["Item ID"].tolist()

def get_columns():
    return ["Item Name", "Category", "Quantity", "Unit", "Price per Unit", "Supplier", "Expiry Date"]

def modify_product(item_id, column, new_value):
    global df
    if item_id in df["Item ID"].values:
        df.loc[df["Item ID"] == item_id, column] = new_value
        if column in ["Quantity", "Price per Unit"]:
            df["Total Cost"] = df["Quantity"].astype(float) * df["Price per Unit"].astype(float)
        df.to_excel(file_path, index=False)
        return df
    return "Item not found"

# Function to remove a product
def remove_product(item_id):
    global df
    df = df[df["Item ID"] != item_id].sort_values(by="Item ID").reset_index(drop=True)
    df.to_excel(file_path, index=False)
    return df

# Function to search products
# Function to search products
def search_product(name):
    global df
    df["Item Name"] = df["Item Name"].astype(str)  # Convert to string to avoid errors
    return df[df["Item Name"].str.contains(name, case=False, na=False)]


# Function to sort by price
def sort_by_price():
    return df.sort_values(by="Price per Unit")

# Function to identify low stock items
# Function to identify low stock items
def low_stock_alert(threshold=10):
    global df
    df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")  # Convert to numeric
    return df[df["Quantity"] < threshold]


# Function to export data to PDF
def generate_pdf():
    pdf_file = "inventory_report.pdf"
    doc = SimpleDocTemplate(pdf_file, pagesize=letter)
    elements = []

    # Define table headers
    data = [["Item ID", "Item Name", "Category", "Quantity", "Unit", "Price per Unit", "Total Cost", "Supplier", "Expiry Date"]]

    # Add inventory data
    for _, row in df.iterrows():
        data.append([
            row["Item ID"], row["Item Name"], row["Category"], row["Quantity"], row["Unit"],
            row["Price per Unit"], row["Total Cost"], row["Supplier"], row["Expiry Date"]
        ])

    # Create table with styles
    table = Table(data)
    table.setStyle(TableStyle([
        ("BACKGROUND", (0, 0), (-1, 0), colors.grey),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
        ("ALIGN", (0, 0), (-1, -1), "CENTER"),
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("BOTTOMPADDING", (0, 0), (-1, 0), 10),
        ("GRID", (0, 0), (-1, -1), 1, colors.black)
    ]))

    elements.append(table)
    doc.build(elements)

    return pdf_file

# Gradio UI with Background Update
with gr.Blocks() as demo:
    gr.Markdown("# 🏢 Warehouse Management System")

    # Apply custom styling for background
    gr.HTML("""
    <style>
      body, .gradio-container {
        background: #528185 !important;
        font-family: 'Poppins', sans-serif !important;
      }
    
      h1 {
        color: black !important;
        font-size: 28px !important;
        font-weight: bold !important;
        text-align: center;
      }
    
      .gradio-container .tab-nav button {
        color: black !important;
        font-size: 18px !important;
        font-weight: bold !important;
      }
    </style>
    """)



    with gr.Tabs():
        with gr.Tab("📋 Inventory"):
            show_button = gr.Button("Show Inventory")
            inventory_output = gr.Dataframe()
            show_button.click(show_inventory, outputs=inventory_output)

        with gr.Tab("➕ Add Product"):
            item_id = gr.Textbox(label="Item ID")
            name = gr.Textbox(label="Item Name")
            category = gr.Textbox(label="Category")
            quantity = gr.Number(label="Quantity")
            unit = gr.Textbox(label="Unit")
            price = gr.Number(label="Price per Unit")
            supplier = gr.Textbox(label="Supplier")
            expiry = gr.Textbox(label="Expiry Date")
            add_button = gr.Button("Add Product")
            add_output = gr.Dataframe()
            add_button.click(add_product, inputs=[item_id, name, category, quantity, unit, price, supplier, expiry], outputs=add_output)

        with gr.Tab("🔄 Modify Product"):
            item_id_dropdown = gr.Dropdown(label="Select Item ID", choices=get_item_ids())
            column_dropdown = gr.Dropdown(label="Select Column to Modify", choices=get_columns())
            new_value_input = gr.Textbox(label="Enter New Value")
            mod_button = gr.Button("Modify")
            mod_output = gr.Dataframe()
            mod_button.click(modify_product, inputs=[item_id_dropdown, column_dropdown, new_value_input], outputs=mod_output)

        with gr.Tab("❌ Remove Product"):
            remove_id = gr.Dropdown(label="Select Item ID", choices=get_item_ids())
            remove_button = gr.Button("Remove")
            remove_output = gr.Dataframe()
            remove_button.click(remove_product, inputs=[remove_id], outputs=remove_output)

        with gr.Tab("🔎 Search Product"):
            search_name = gr.Textbox(label="Search by Name")
            search_button = gr.Button("Search")
            search_output = gr.Dataframe()
            search_button.click(search_product, inputs=[search_name], outputs=search_output)

        with gr.Tab("💰 Sort by Price"):
            sort_button = gr.Button("Sort")
            sort_output = gr.Dataframe()
            sort_button.click(sort_by_price, outputs=sort_output)

        with gr.Tab("⚠ Low Stock Alert"):
            threshold = gr.Number(label="Threshold (Default: 10)", value=10)
            low_stock_button = gr.Button("Check Low Stock")
            low_stock_output = gr.Dataframe()
            low_stock_button.click(low_stock_alert, inputs=[threshold], outputs=low_stock_output)

        with gr.Tab("📄 Export to PDF"):
            pdf_button = gr.Button("Download PDF")
            pdf_output = gr.File()
            pdf_button.click(generate_pdf, outputs=pdf_output)

demo.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "D:\Anaconda\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\Lib\site-packages\gradio\blocks.py", line 2113, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\Lib\site-packages\gradio\blocks.py", line 1919, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Anaconda\Lib\site-packages\gradio\components\dataframe.py", line 421, in postprocess
    headers = self.get_headers(value) or self